# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
# import libraries
import re
import pandas as pd

from sqlalchemy import create_engine
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

from sklearn.pipeline import Pipeline

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table(engine.table_names()[0],con=engine)
X = df.message
y = df.iloc[:,5:]

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    
    # normalize
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize sentence
    token = word_tokenize(text)
    # remove stopwords and lemmatize 
    words = [WordNetLemmatizer().lemmatize(w) for w in token if w not in stopwords.words('english')]

    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
def performance(model, X_test, y_test):
    y_pred = model.predict(X_test)
    for i, col in enumerate(y_test):
        print(col)
        print(classification_report(y_test[col], y_pred[:, i]))
performance(pipeline, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.64      0.46      0.53      1548
          1       0.84      0.92      0.88      4964
          2       0.25      0.31      0.28        42

avg / total       0.79      0.80      0.79      6554

request
             precision    recall  f1-score   support

          0       0.89      0.97      0.93      5396
          1       0.79      0.45      0.57      1158

avg / total       0.87      0.88      0.87      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6531
          1       0.00      0.00      0.00        23

avg / total       0.99      1.00      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.75      0.85      0.80      3812
          1       0.74      0.60      0.67      2742

avg / total       0.75      0.75      0.74      6554

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [17]:
# Check the list of available parameters with `estimator.get_params().keys()`.

list(pipeline.get_params().keys())

['memory',
 'steps',
 'vect',
 'tfidf',
 'clf',
 'vect__analyzer',
 'vect__binary',
 'vect__decode_error',
 'vect__dtype',
 'vect__encoding',
 'vect__input',
 'vect__lowercase',
 'vect__max_df',
 'vect__max_features',
 'vect__min_df',
 'vect__ngram_range',
 'vect__preprocessor',
 'vect__stop_words',
 'vect__strip_accents',
 'vect__token_pattern',
 'vect__tokenizer',
 'vect__vocabulary',
 'tfidf__norm',
 'tfidf__smooth_idf',
 'tfidf__sublinear_tf',
 'tfidf__use_idf',
 'clf__estimator__bootstrap',
 'clf__estimator__class_weight',
 'clf__estimator__criterion',
 'clf__estimator__max_depth',
 'clf__estimator__max_features',
 'clf__estimator__max_leaf_nodes',
 'clf__estimator__min_impurity_decrease',
 'clf__estimator__min_impurity_split',
 'clf__estimator__min_samples_leaf',
 'clf__estimator__min_samples_split',
 'clf__estimator__min_weight_fraction_leaf',
 'clf__estimator__n_estimators',
 'clf__estimator__n_jobs',
 'clf__estimator__oob_score',
 'clf__estimator__random_state',
 'clf__estimat

In [18]:
from sklearn.model_selection import GridSearchCV

parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
        'vect__min_df': (0.05,0.1,0.15),
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 100],
        'clf__estimator__max_depth':[10,20,30]}

gs_clf = GridSearchCV(estimator = pipeline, param_grid=parameters,scoring='f1_weighted', n_jobs=-1, verbose=1)

In [22]:
import numpy as np
np.array(y_train)

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 1, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [26]:
gs_clf.fit(X_train,y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits


ValueError: Classification metrics can't handle a mix of multiclass-multioutput and multilabel-indicator targets

In [ ]:
best_parameters = grid_clf.best_params_
best_accuracy = grid_clf.best_score_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
gs_clf.predict(X_test, y_test)


ValueError: Invalid parameter max_depth for estimator MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1). Check the list of available parameters with `estimator.get_params().keys()`.

In [ ]:
performance(cv, X_test, y_test)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.